In [1]:
from math import log
import numpy as np
import operator

In [2]:
def loaddata():
    dataSet = [[0, 0,0,0, 'no'],
               [0, 0,0,1,'no'],
               [0, 1,0,1, 'yes'],
               [0, 1,1,0, 'yes'],
               [0, 0,0,0, 'no'],
               [1, 0,0,0, 'no'],
               [1, 0,0,1, 'no'],
               [1, 1,1,1, 'yes'],
               [1, 0,1,2, 'yes'],
               [1, 0,1,2, 'yes'],
               [2, 0,1,2, 'yes'],
               [2, 0,1,1, 'yes'],
               [2, 1,0,1, 'yes'],
               [2, 1,0,2, 'yes'],
               [2, 0,0,0,'no']]
    feature_name = ['age','job','house','credit']
    return dataSet, feature_name

In [3]:
def entropy(dataSet):
    #数据集条数
    m = len(dataSet)
    #标签不同类别的计数字典
    labelCounts = {}
    #循环数据集
    for featVec in dataSet:
        #取标签
        currentLabel = featVec[-1]
        #如果字典中不存在则值为0，否则值加1
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    #保存最终的熵值
    e = 0.0
    #根据公式计算熵
    for key in labelCounts:
        prob = float(labelCounts[key])/m
        e -= prob * log(prob,2)
    return e

In [4]:
dataSet, feature_name = loaddata()
entropy(dataSet)

0.9709505944546686

In [5]:
def splitDataSet(dataSet, axis, value):
    #按轴和值划分好的数据集
    retDataSet = []
    #循环数据集
    for featVec in dataSet:
        #当前数据按轴取出的数据符合传入的value值
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [6]:
splitDataSet(dataSet,2,0)

[[0, 0, 0, 'no'],
 [0, 0, 1, 'no'],
 [0, 1, 1, 'yes'],
 [0, 0, 0, 'no'],
 [1, 0, 0, 'no'],
 [1, 0, 1, 'no'],
 [2, 1, 1, 'yes'],
 [2, 1, 2, 'yes'],
 [2, 0, 0, 'no']]

In [7]:
def chooseBestFeature(dataSet):
    #特征数
    n = len(dataSet[0]) - 1
    #计数整个数据集的熵
    baseEntropy = entropy(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    #遍历每个特征
    for i in range(n):
        #获取当前特征的所有值
        featList = [example[i] for example in dataSet]
        #当前特征的可能取值
        uniqueVals = set(featList)
        #定义一临时变量保存当前的条件熵
        newEntropy = 0.0
        #循环每一个可能的取值
        for value in uniqueVals:
            #按该值进行数据集的划分
            subDataSet = splitDataSet(dataSet, i, value)
            #计算条件熵（2行代码）
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * entropy(subDataSet)
        #计算信息增益
        infoGain = baseEntropy - newEntropy
        #保存当前最大的信息增益及对应的特征
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    #返回最优特征
    return bestFeature

In [9]:
chooseBestFeature(dataSet)

2

In [16]:
def classVote(classList):
    #定义一字典，记录每个标签对应的个数
    classCount={}
    #循环计数
    for vote in classList:
        if vote not in classCount.keys():
            classCount[ vote]=0
        classCount[vote] += 1

    #排序
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)

    return  sortedClassCount[0][0]

In [19]:
classList=np.array(['yes','no','yes','no','yes'])
classVote(classList)

np.str_('yes')

In [12]:
def trainTree(dataSet,feature_name):
    classList=[example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return classVote(classList)
    bestFeat=chooseBestFeature(dataSet)
    bestFeatName=feature_name[bestFeat]
    myTree={bestFeatName:{}}
    featValues=[example[bestFeat] for example in dataSet]
    uniqueVals=set(featValues)
    for value in uniqueVals:
        sub_feature_name=feature_name[:]
        myTree[bestFeatName][value]=trainTree(splitDataSet(dataSet,bestFeat,value),sub_feature_name)
    return myTree

In [21]:
myDat,feature_name=loaddata()
myTree=trainTree(myDat,feature_name)
print(myTree)

{'house': {0: {'job': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


In [22]:
def predict(inputTree,featLabels,testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict):
        classLabel = predict(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

In [23]:
print(predict(myTree,feature_name,[1,1,0,1]))

yes
